In [10]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [1]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
from sklearn.model_selection import KFold
from modules.data_structures.model_dataset import ModelDataset
from model_binaries.utils.binary_utils import save_entity, load_entity, cross_val_train, compute_f1

2025-01-29 10:07:20.706772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations

## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v0/model_v0_000.yaml'
yaml_paths = [yaml_path0]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()

Load Train into Memory

In [3]:
# File path to load train, test, and val datasets
train_test_val_folder_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v0/scaler"

In [4]:
train_dataset = load_entity(train_test_val_folder_path, "scaled_train.pkl")

Create and Train Models Using KFold Cross Validation

In [5]:
# === STEP 4: MODEL MANAGER
yamls = [yaml_path0]
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)

In [6]:
avg_metrics = cross_val_train(model_manager=model_manager, yamls=yamls, train_dataset=train_dataset, n_splits=3)

INFO:root:Training model '0c03eef430f4878e2d1dcca4e08bbffe' for 20 epochs with batch size 32.


Epoch 1/20
478/478 [==============================] - 3s 4ms/step - loss: 0.6771 - accuracy: 0.4615 - precision: 0.6154 - recall: 0.2456 - auc: 0.5196 - val_loss: 0.6724 - val_accuracy: 0.4442 - val_precision: 0.7472 - val_recall: 0.0754 - val_auc: 0.5770
Epoch 2/20
478/478 [==============================] - 1s 3ms/step - loss: 0.6703 - accuracy: 0.5003 - precision: 0.6547 - recall: 0.3333 - auc: 0.5620 - val_loss: 0.6742 - val_accuracy: 0.5789 - val_precision: 0.6280 - val_recall: 0.6874 - val_auc: 0.5890
Epoch 3/20
478/478 [==============================] - 2s 4ms/step - loss: 0.6667 - accuracy: 0.5186 - precision: 0.6610 - recall: 0.3867 - auc: 0.5778 - val_loss: 0.6671 - val_accuracy: 0.5037 - val_precision: 0.6896 - val_recall: 0.2755 - val_auc: 0.5911
Epoch 4/20
478/478 [==============================] - 2s 4ms/step - loss: 0.6650 - accuracy: 0.5270 - precision: 0.6664 - recall: 0.4054 - auc: 0.5847 - val_loss: 0.6690 - val_accuracy: 0.5517 - val_precision: 0.6505 - val_recall: 0

INFO:root:Model '0c03eef430f4878e2d1dcca4e08bbffe': Finished training.


Model saved successfully at: models/0c03eef430f4878e2d1dcca4e08bbffe/model_weights_0c03eef430f4878e2d1dcca4e08bbffe.pth


INFO:root:Training model '0c03eef430f4878e2d1dcca4e08bbffe' for 20 epochs with batch size 32.


Epoch 1/20
478/478 [==============================] - 2s 3ms/step - loss: 0.6765 - accuracy: 0.4716 - precision: 0.6226 - recall: 0.2617 - auc: 0.5338 - val_loss: 0.6728 - val_accuracy: 0.4179 - val_precision: 0.6912 - val_recall: 0.0331 - val_auc: 0.5759
Epoch 2/20
478/478 [==============================] - 1s 3ms/step - loss: 0.6699 - accuracy: 0.5100 - precision: 0.6594 - recall: 0.3482 - auc: 0.5711 - val_loss: 0.6668 - val_accuracy: 0.5227 - val_precision: 0.6642 - val_recall: 0.3947 - val_auc: 0.5784
Epoch 3/20
478/478 [==============================] - 1s 3ms/step - loss: 0.6664 - accuracy: 0.5237 - precision: 0.6645 - recall: 0.3866 - auc: 0.5842 - val_loss: 0.6665 - val_accuracy: 0.5107 - val_precision: 0.6626 - val_recall: 0.3563 - val_auc: 0.5783
Epoch 4/20
478/478 [==============================] - 1s 3ms/step - loss: 0.6649 - accuracy: 0.5339 - precision: 0.6731 - recall: 0.4060 - auc: 0.5909 - val_loss: 0.6679 - val_accuracy: 0.4973 - val_precision: 0.6769 - val_recall: 0

INFO:root:Model '0c03eef430f4878e2d1dcca4e08bbffe': Finished training.
INFO:root:Training model '0c03eef430f4878e2d1dcca4e08bbffe' for 20 epochs with batch size 32.


Model saved successfully at: models/0c03eef430f4878e2d1dcca4e08bbffe/model_weights_0c03eef430f4878e2d1dcca4e08bbffe.pth
Epoch 1/20
478/478 [==============================] - 3s 4ms/step - loss: 0.6755 - accuracy: 0.4936 - precision: 0.6330 - recall: 0.3337 - auc: 0.5447 - val_loss: 0.6862 - val_accuracy: 0.4117 - val_precision: 0.6900 - val_recall: 0.0152 - val_auc: 0.5567
Epoch 2/20
478/478 [==============================] - 2s 4ms/step - loss: 0.6680 - accuracy: 0.5238 - precision: 0.6681 - recall: 0.3805 - auc: 0.5798 - val_loss: 0.6681 - val_accuracy: 0.4950 - val_precision: 0.6772 - val_recall: 0.2842 - val_auc: 0.5756
Epoch 3/20
478/478 [==============================] - 2s 4ms/step - loss: 0.6651 - accuracy: 0.5305 - precision: 0.6707 - recall: 0.3986 - auc: 0.5885 - val_loss: 0.6680 - val_accuracy: 0.5005 - val_precision: 0.6804 - val_recall: 0.2979 - val_auc: 0.5781
Epoch 4/20
478/478 [==============================] - 2s 3ms/step - loss: 0.6618 - accuracy: 0.5416 - precision:

INFO:root:Model '0c03eef430f4878e2d1dcca4e08bbffe': Finished training.


Model saved successfully at: models/0c03eef430f4878e2d1dcca4e08bbffe/model_weights_0c03eef430f4878e2d1dcca4e08bbffe.pth


Get Averaged Metrics for KFold

In [10]:
avg_metrics

{'loss': 0.5827763477961222,
 'accuracy': 0.6602229277292887,
 'precision': 0.7658493320147196,
 'recall': 0.6112801631291708,
 'auc': 0.722386101881663,
 'val_loss': 0.7392456531524658,
 'val_accuracy': 0.5152283509572347,
 'val_precision': 0.6273205280303955,
 'val_recall': 0.44025274117787677,
 'val_auc': 0.5412235657374064}

In [9]:
f1 = compute_f1(avg_metrics["val_precision"], avg_metrics["val_recall"])
print(f1)

0.5173969600556849


Launch Tensorboard

In [15]:
%tensorboard --logdir .logs/fit/

UsageError: Line magic function `%tensorboard` not found.
